In [0]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

Socrata Open Data API (SODA) 

In [0]:
#Read in csv
df=pd.read_csv('https://data.cityofnewyork.us/resource/43nn-pn8j.csv?$limit=500000')

In [0]:
df=pd.DataFrame(df)

In [4]:
df.groupby("boro").head(10)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,41636291,NIFOROS CORNER,Queens,3560,11TH ST,11106.0,7185455971,Delicatessen,2018-03-29T00:00:00.000,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-u...",Y,13.0,A,2018-03-29T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.761827,-73.939536,401.0,26.0,8500.0,4003965.0,4.003300e+09,QN68
1,50095507,CASA ORA,Brooklyn,148,MESEROLE STREET,11206.0,9293970281,"Latin (Cuban, Dominican, Puerto Rican, South &...",2019-10-02T00:00:00.000,Violations were cited in the following area(s).,20D,“Choking first aid” poster not posted. “Alcoho...,N,NaN,NaN,NaN,2020-05-05T06:01:20.000,Administrative Miscellaneous / Initial Inspection,40.708010,-73.943794,301.0,34.0,50500.0,3071115.0,3.030520e+09,BK90
2,50082813,WOK 88,Manhattan,1570,3 AVENUE,10128.0,2128288688,Chinese,2019-10-15T00:00:00.000,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,N,18.0,NaN,NaN,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.780272,-73.952933,108.0,5.0,14802.0,1048085.0,1.015170e+09,MN40
3,50064340,OPTIMISTIC CAFE,Manhattan,260,WEST 39 STREET,10018.0,3473719175,Café/Coffee/Tea,2019-03-15T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,7.0,A,2019-03-15T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Re-inspection,40.754810,-73.989984,105.0,3.0,11300.0,1014490.0,1.007880e+09,MN17
4,40424972,JACKIE'S WEST INDIAN BAKERY,Bronx,1203,EAST 233 STREET,10466.0,7189942541,Caribbean,2019-02-14T00:00:00.000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,N,13.0,A,2019-02-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.888745,-73.842809,212.0,12.0,42600.0,2066569.0,2.049540e+09,BX62
5,50088941,EL MALECON RESTAURANT,Brooklyn,8727,4 AVENUE,11209.0,3479097132,Spanish,2019-04-01T00:00:00.000,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Y,28.0,NaN,NaN,2020-05-05T06:01:20.000,Pre-permit (Operational) / Initial Inspection,40.621356,-74.028929,310.0,43.0,16000.0,3153748.0,3.060500e+09,BK31
6,50010706,SHAXIAN DELICACIES,Brooklyn,811,49 STREET,11220.0,7186665350,Chinese,2017-04-21T00:00:00.000,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Y,12.0,A,2017-04-21T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.641380,-74.002630,312.0,38.0,10800.0,3012805.0,3.007780e+09,BK34
7,41645448,BOCBOC CHICKEN DELICIOUS,Queens,90-15,QUEENS BOULEVARD,11373.0,7182718098,Asian,2018-05-14T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,12.0,A,2018-05-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.733840,-73.871577,404.0,25.0,68300.0,4045999.0,4.018600e+09,QN29
8,50063594,MI ESTRELLA NY BAKERY,Brooklyn,268,WYCKOFF AVENUE,11237.0,7186289250,Bakery,2018-10-25T00:00:00.000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,N,6.0,A,2018-10-25T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.700184,-73.912283,304.0,37.0,43900.0,3076369.0,3.033370e+09,BK77
9,50045479,FORTUNE HAWAII,Staten Island,900,HYLAN BOULEVARD,10305.0,7189818383,Chinese,2017-12-13T00:00:00.000,Violations were cited in the following area(s).,04C,Food worker does not use proper utensil to eli...,Y,20.0,NaN,NaN,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.598283,-74.072674,502.0,50.0,6400.0,5095402.0,5.032660e+09,SI14


In [5]:
#Check the size of dataframe
df.shape

(389802, 26)

In [6]:
#Take a look at the first 5 rows
df.head(5)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,41636291,NIFOROS CORNER,Queens,3560,11TH ST,11106.0,7185455971,Delicatessen,2018-03-29T00:00:00.000,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-u...",Y,13.0,A,2018-03-29T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.761827,-73.939536,401.0,26.0,8500.0,4003965.0,4.003300e+09,QN68
1,50095507,CASA ORA,Brooklyn,148,MESEROLE STREET,11206.0,9293970281,"Latin (Cuban, Dominican, Puerto Rican, South &...",2019-10-02T00:00:00.000,Violations were cited in the following area(s).,20D,“Choking first aid” poster not posted. “Alcoho...,N,NaN,NaN,NaN,2020-05-05T06:01:20.000,Administrative Miscellaneous / Initial Inspection,40.708010,-73.943794,301.0,34.0,50500.0,3071115.0,3.030520e+09,BK90
2,50082813,WOK 88,Manhattan,1570,3 AVENUE,10128.0,2128288688,Chinese,2019-10-15T00:00:00.000,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,N,18.0,NaN,NaN,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.780272,-73.952933,108.0,5.0,14802.0,1048085.0,1.015170e+09,MN40
3,50064340,OPTIMISTIC CAFE,Manhattan,260,WEST 39 STREET,10018.0,3473719175,Café/Coffee/Tea,2019-03-15T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,7.0,A,2019-03-15T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Re-inspection,40.754810,-73.989984,105.0,3.0,11300.0,1014490.0,1.007880e+09,MN17
4,40424972,JACKIE'S WEST INDIAN BAKERY,Bronx,1203,EAST 233 STREET,10466.0,7189942541,Caribbean,2019-02-14T00:00:00.000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,N,13.0,A,2019-02-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.888745,-73.842809,212.0,12.0,42600.0,2066569.0,2.049540e+09,BX62


In [7]:
print(df.columns) 

Index(['camis', 'dba', 'boro', 'building', 'street', 'zipcode', 'phone',
       'cuisine_description', 'inspection_date', 'action', 'violation_code',
       'violation_description', 'critical_flag', 'score', 'grade',
       'grade_date', 'record_date', 'inspection_type', 'latitude', 'longitude',
       'community_board', 'council_district', 'census_tract', 'bin', 'bbl',
       'nta'],
      dtype='object')


In [8]:
# checking missing data in data 
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
grade_date,194782,49.969472
grade,192161,49.297079
score,16297,4.180841
critical_flag,8717,2.236264
violation_description,8717,2.236264


In [0]:
#remove any rows that have no grade, no business name (dba), or location information
df = df.dropna(subset=['grade','dba','latitude','longitude','zipcode','street'])

In [10]:
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
grade_date,2589,1.331687
critical_flag,1648,0.847671
violation_description,1648,0.847671
violation_code,897,0.461384
bin,731,0.376000


## now data is good without NaN



In [0]:
df.columns = df.columns.str.replace("CUISINE DESCRIPTION","CUISINE")
df.columns = df.columns.str.replace(" ","_")# remove space 

In [0]:
df["dba"].replace("'","", inplace=True) # Remove apostrophe
df["dba"].replace(" ?\(.+\)", "", regex=True, inplace=True) # Remove values in parenthesis
df["dba"].replace(" ?#.*", "", regex=True, inplace=True) # Remove # followed by some string


In [13]:
len(df)

194415

In [0]:
clean_df=df# make a copy of the clean dataframe, then don't need to run over again 

In [15]:
clean_df.grade.replace(['A', 'B','C','N','P','Z'], ['ABC','BC','C','Z','Z','Z'], inplace=True) # replace grade with other formats 
clean_df.head(5)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,41636291,NIFOROS CORNER,Queens,3560,11TH ST,11106.0,7185455971,Delicatessen,2018-03-29T00:00:00.000,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-u...",Y,13.0,ABC,2018-03-29T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.761827,-73.939536,401.0,26.0,8500.0,4003965.0,4.003300e+09,QN68
3,50064340,OPTIMISTIC CAFE,Manhattan,260,WEST 39 STREET,10018.0,3473719175,Café/Coffee/Tea,2019-03-15T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,7.0,ABC,2019-03-15T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Re-inspection,40.754810,-73.989984,105.0,3.0,11300.0,1014490.0,1.007880e+09,MN17
4,40424972,JACKIE'S WEST INDIAN BAKERY,Bronx,1203,EAST 233 STREET,10466.0,7189942541,Caribbean,2019-02-14T00:00:00.000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,N,13.0,ABC,2019-02-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.888745,-73.842809,212.0,12.0,42600.0,2066569.0,2.049540e+09,BX62
6,50010706,SHAXIAN DELICACIES,Brooklyn,811,49 STREET,11220.0,7186665350,Chinese,2017-04-21T00:00:00.000,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Y,12.0,ABC,2017-04-21T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.641380,-74.002630,312.0,38.0,10800.0,3012805.0,3.007780e+09,BK34
7,41645448,BOCBOC CHICKEN DELICIOUS,Queens,90-15,QUEENS BOULEVARD,11373.0,7182718098,Asian,2018-05-14T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,12.0,ABC,2018-05-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.733840,-73.871577,404.0,25.0,68300.0,4045999.0,4.018600e+09,QN29


In [0]:
top20=clean_df.groupby(['cuisine_description']).count().sort_values(by='dba', ascending=False).head(20)

In [19]:
print(top20.index)

Index(['American', 'Chinese', 'Café/Coffee/Tea', 'Pizza', 'Mexican', 'Italian',
       'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
       'Japanese', 'Bakery', 'Caribbean', 'Spanish', 'Pizza/Italian',
       'Chicken', 'Donuts', 'Indian', 'Asian', 'Delicatessen', 'Jewish/Kosher',
       'Hamburgers', 'Thai'],
      dtype='object', name='cuisine_description')


In [0]:
top20list=['American', 'Chinese', 'Café/Coffee/Tea', 'Pizza', 'Mexican', 'Italian',
       'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
       'Japanese', 'Bakery', 'Caribbean', 'Spanish', 'Pizza/Italian',
       'Chicken', 'Donuts', 'Indian', 'Asian', 'Delicatessen', 'Jewish/Kosher',
       'Hamburgers', 'Thai']

In [21]:
df1=clean_df[clean_df['cuisine_description'].isin(top20list)] # filter, select cusine under top20
df1=pd.DataFrame(df1) #make sure it's a dataframe
df1.head()
 

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,grade_date,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,41636291,NIFOROS CORNER,Queens,3560,11TH ST,11106.0,7185455971,Delicatessen,2018-03-29T00:00:00.000,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-u...",Y,13.0,ABC,2018-03-29T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.761827,-73.939536,401.0,26.0,8500.0,4003965.0,4.003300e+09,QN68
3,50064340,OPTIMISTIC CAFE,Manhattan,260,WEST 39 STREET,10018.0,3473719175,Café/Coffee/Tea,2019-03-15T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,7.0,ABC,2019-03-15T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Re-inspection,40.754810,-73.989984,105.0,3.0,11300.0,1014490.0,1.007880e+09,MN17
4,40424972,JACKIE'S WEST INDIAN BAKERY,Bronx,1203,EAST 233 STREET,10466.0,7189942541,Caribbean,2019-02-14T00:00:00.000,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,N,13.0,ABC,2019-02-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.888745,-73.842809,212.0,12.0,42600.0,2066569.0,2.049540e+09,BX62
6,50010706,SHAXIAN DELICACIES,Brooklyn,811,49 STREET,11220.0,7186665350,Chinese,2017-04-21T00:00:00.000,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Y,12.0,ABC,2017-04-21T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.641380,-74.002630,312.0,38.0,10800.0,3012805.0,3.007780e+09,BK34
7,41645448,BOCBOC CHICKEN DELICIOUS,Queens,90-15,QUEENS BOULEVARD,11373.0,7182718098,Asian,2018-05-14T00:00:00.000,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Y,12.0,ABC,2018-05-14T00:00:00.000,2020-05-05T06:01:20.000,Cycle Inspection / Initial Inspection,40.733840,-73.871577,404.0,25.0,68300.0,4045999.0,4.018600e+09,QN29


In [23]:
len(df1)

157228

In [0]:
#convert longitude and latitude to float to do precise calculations
df1["longitude"] = df1["longitude"].astype(float)
df1["latitude"] = df1["latitude"].astype(float)

# User Input

In [0]:
#install ipywidgets package
!pip install ipywidgets  
!jupyter nbextension enable --py widgetsnbextension


In [0]:
#install ipyleaflet package
!pip install ipyleaflet

In [0]:
import ipywidgets as widgets #Package for python widgets
from ipyleaflet import Map,basemaps,Marker #Package enabling interactive maps in the Jupyter notebook
from IPython.display import display

#### Get the User Coordinates by UDF from Shreeshma


In [0]:
import requests
import json
def display_ip():
   """  Function To Print GeoIP Latitude & Longitude """
   ip_request = requests.get('https://get.geojs.io/v1/ip.json')
   my_ip = ip_request.json()['ip']
   #print(my_ip)
   geo_request = requests.get('https://get.geojs.io/v1/ip/geo/' +my_ip + '.json')
   geo_data = geo_request.json()
   return(geo_data['latitude'],geo_data['longitude'])
 
location = display_ip()
my_latitude = float(location[0])
my_longitude = float(location[1])


In [35]:
print(my_latitude)
print(my_longitude)

38.6583
-77.2481


In [0]:
#Initialize a 5*2 grid layout
grid = widgets.GridspecLayout(5, 2, height='900px')

#Initialize a Map
center = [my_latitude, my_longitude]
zoom = 18
m = Map(basemap=basemaps.OpenStreetMap.Mapnik,center=center, zoom=zoom)

#Initialize a dropdown box, and make the options to be the top20list 
dropdown_cuisine = widgets.Dropdown(options = top20list, description = 'Cuisine Type:')

#Initialize a radio buttons for grade selection
radio_grade = widgets.RadioButtons(options=[('A', 'ABC'), ('B', 'BC'), ('C', 'C')], \
                                   description='Grade:', \
                                   disabled=False)

#Initialize an output widget 
output = widgets.Output()

#Add each component to the grid layout
grid[0,0] = dropdown_cuisine
grid[0,1] = radio_grade
grid[1,:] = m
grid[2:4,:] = output

In [0]:
#Define the event handler for dropdown_cuisine
def dropdown_cuisine_eventhandler(change):
    common_filtering(change.new, radio_grade.value, m) #change.new is the lastest value from dropdown_cuisine

#Define the event handler for radio_grade
def radio_grade_eventhandler(change):
    common_filtering(dropdown_cuisine.value, change.new, m) #change.new is the lastest value from radio_grade
    
#Define the event handler for map
def map_eventhandler(change):
    common_filtering(dropdown_cuisine.value, radio_grade.value, change.value) #change.new is the lastest value from map


#Define the filtering function (from Karim's algorithm) 
def common_filtering(cuisine, grade, m):
    #clear the previous selection output on each iteration
    output.clear_output()
    
    #clear the previous markers on the map
    base_layer = m.layers[0]
    m.clear_layers()
    m.add_layer(base_layer)
    
    #Assign the values based on user's choice
    restaurant_choice = cuisine
    grade_choice = grade
    
    #----------------------------------------ALGORITHM STARTS HERE----------------------------------------#
    #restaurant have multiple times they were test, keep only last grade
    filtered_df = df1.sort_values('grade_date').drop_duplicates('dba',keep='last') 

    #filter by restaurant choices
    filtered_df=filtered_df.loc[df1['cuisine_description'] == restaurant_choice] 

    #filter by grade of interest
    filtered_df=filtered_df.loc[df1['grade'] == grade_choice]

    #calculate proximity: use absolute value delta of user's lat / long and the restaurant's; convert to miles where 1 degree = 68.703 miles
    filtered_df["distance"] = (abs(filtered_df['latitude'] - my_latitude) + abs(filtered_df['longitude'] - my_longitude)) * 68.703

    # replace grades back the grades that are understandable
    filtered_df.grade.replace(['ABC','BC','C'], ['A', 'B','C'], inplace=True)

    #clean up what columns to keep for results
    results_df = filtered_df.loc[:,["dba","cuisine_description","building","street","boro","grade","grade_date","latitude","longitude","distance"]].copy()

    #sort by total distance column, smallest at the top
    results_df = results_df.sort_values(["distance"],ascending=[True])

    #get the top 5 results to display
    results_df = results_df.head(5)
    
    #----------------------------------------ALGORITHM ENDS HERE----------------------------------------#

    for index, row in results_df.iterrows():        
        center = (row['latitude'],row['longitude'])
        marker = Marker(location = center, draggable = False, title = row['dba'])
        m.add_layer(marker)
    
    #display the result dataframe
    with output:
        display(results_df)

In [0]:
#bind the event handlers to the dropdown/radio selection and map by the observe() function
#so that every time user makes new selections, it will invoke dropdown_cuisine_eventhandler/radio_grade_eventhandler/map_eventhandler
dropdown_cuisine.observe(dropdown_cuisine_eventhandler, names='value')
radio_grade.observe(radio_grade_eventhandler, names='value')
m.observe(map_eventhandler, names='value')


In [39]:
#Display the grid layout
display(grid)

GridspecLayout(children=(Dropdown(description='Cuisine Type:', layout=Layout(grid_area='widget001'), options=(…

# Algorithm

In [0]:
#for testing, we temporily asks for a choice from notebook input
restaurant_choice = input("Type 1 for Italian or 2 for Chinese:")  #fake a user interface for now
if restaurant_choice == "1":
  restaurant_choice = "Italian"
else:
  restaurant_choice = "Chinese"  
grade_choice = "BC" #say that I just want B grades
my_latitude = 40.728453 #float(input("Type latitude:"))
my_longitude = -73.979612 #float(input("Type longitude:"))

In [0]:
#restaurant have multiple times they were test, keep only last grade
filtered_df = df1.sort_values('grade_date').drop_duplicates('dba',keep='last') 

#filter by restaurant choices
filtered_df=filtered_df.loc[df1['cuisine_description'] == restaurant_choice] 

#filter by grade of interest
filtered_df=filtered_df.loc[df1['grade'] == grade_choice]

#calculate proximity: use absolute value delta of user's lat / long and the restaurant's; convert to miles where 1 degree = 68.703 miles
filtered_df["distance"] = (abs(filtered_df['latitude'] - my_latitude) + abs(filtered_df['longitude'] - my_longitude)) * 68.703

# replace grades back the grades that are understandable
filtered_df.grade.replace(['ABC','BC','C'], ['A', 'B','C'], inplace=True)

#clean up what columns to keep for results
results_df = filtered_df.loc[:,["dba","cuisine_description","building","street","boro","grade","grade_date","latitude","longitude","distance"]].copy()

#sort by total distance column, smallest at the top
results_df = results_df.sort_values(["distance"],ascending=[True])

#get the top 5 results to display
results_df = results_df.head(5)
results_df

# Export to CSV in Colab

In [0]:
from google.colab import files
results_df.to_csv('results.csv') 
files.download('results.csv')